In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import lightning as pl

from dataset import TransactionDataset, transaction_collate_fn

In [9]:
df = pd.read_csv('data/rosbank/train.csv')
df['TRDATETIME'] = pd.to_datetime(df['TRDATETIME'], format='%d%b%y:%H:%M:%S')
df = df.rename(columns={'cl_id':'client_id', 'MCC':'small_group', 'amount':'amount_rur'})
df.head()

,PERIOD,client_id,small_group,channel_type,currency,TRDATETIME,amount_rur,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,2017-10-21 00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,2017-10-12 12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,2017-12-05 00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,2017-10-21 00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,2017-10-24 13:14:24,36562.0,C2C_OUT,0,0.0


In [10]:
mcc_to_id = {mcc: i+1 for i, mcc in enumerate(df['small_group'].unique())}

df['amount_rur_bin'] = 1 + KBinsDiscretizer(10, encode='ordinal', subsample=None).fit_transform(df[['amount_rur']]).astype('int')
df['small_group'] = df['small_group'].map(mcc_to_id)

In [15]:
ds = TransactionDataset(
    df, 
    id_col="client_id", 
    dt_col="TRDATETIME", 
    cat_cols=["small_group", "amount_rur_bin"],
    min_length=10,
    max_length=100
)

383 sequences were filtered


In [47]:
batch = [ds[0]]

In [1]:
class TransactionEncoder(nn.Module):
    def __init__(self, embedding_dict, linear_proj=None):
        super().__init__()

        self.features = embedding_dict.keys()
        self.embeddings = nn.ModuleDict({key: nn.Embedding(vocab, dim) for key, (vocab, dim) in embedding_dict.items()})
        self.linear_proj = nn.Identity()
        if linear_proj is not None:
            self.embedding_dim = linear_proj
            self.linear_proj = nn.Linear(sum([dim for key, (vocab, dim) in embedding_dict.items()]), linear_proj)
        else:
            self.embedding_dim = sum([dim for key, (vocab, dim) in embedding_dict.items()])

    def forward(self, x):
        embeddings = [self.embeddings[key](x[key]) for key in self.features]
        proj = self.linear_proj(torch.cat(embeddings, dim=2))
        return proj

NameError: name 'nn' is not defined

In [56]:
a = TransactionEncoder(embedding_dict={"small_group": (500, 32), "amount_rur_bin": (11, 32)}, linear_proj=128)

In [57]:
a(transaction_collate_fn(batch)).shape

torch.Size([1, 76, 128])

In [55]:
a.requires_grad_(False)

TransactionEncoder(
  (embeddings): ModuleDict(
    (small_group): Embedding(500, 32)
    (amount_rur_bin): Embedding(11, 32)
  )
  (linear_proj): Identity()
)

In [ ]:
class TransformerMode(nn.Module):
    def __init__(self, encoder):
        self.encoder = encoder
        self.decoder_layer = ...
        self.decoder = ...
        self.head = ...

    def forward(self, x):
